In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [3]:
data=pd.read_excel("/content/drive/MyDrive/Bert/Release_Notes_v0_RawData.xlsx")

In [4]:
data

,ID,Project_Name,Release_Number,Date,Label,Type,Release_Note
0,1.0,Mozilla Firefox,86.0,2021-02-23,new,new,Firefox now supports simultaneously watching m...
1,2.0,Mozilla Firefox,86.0,2021-02-23,fixed,fixed,Reader mode now works with local HTML pages. U...
2,3.0,Mozilla Firefox,86.0,2021-02-23,changed,changed,"On Linux and Android, the protection to mitiga..."
3,4.0,Mozilla Firefox,86.0,2021-02-23,other,developer,Inactive CSS tool is now showing a warning whe...
4,5.0,Mozilla Firefox,85.0.2,2021-02-09,fixed,fixed,Fixed a deadlock during startup (bug 1679933)
...,...,...,...,...,...,...,...
795,796.0,Slack,2.8.0,2017-09-11,fixed,fixed,Fixed: A few rare crashes when making a call a...
796,797.0,Slack,2.7.1,2017-08-17,fixed,fixed,You're nearly finished signing in when suddenl...
797,798.0,Slack,2.7.0,2017-08-01,new,new,File downloads are now pausable and – in perha...
798,799.0,Slack,2.7.0,2017-08-01,fixed,fixed,"On Windows 7, notifications have been made mor..."


In [5]:
data["Label"].unique()

array(['new', 'fixed', 'changed', 'other', 'unresolved'], dtype=object)

In [26]:
data['labels'] = pd.factorize(data.Label)[0]

In [35]:
data.head()

,ID,Project_Name,Release_Number,Date,Label,Type,Release_Note,labels
0,1.0,Mozilla Firefox,86.0,2021-02-23,new,new,Firefox now supports simultaneously watching m...,0
1,2.0,Mozilla Firefox,86.0,2021-02-23,fixed,fixed,Reader mode now works with local HTML pages. U...,1
2,3.0,Mozilla Firefox,86.0,2021-02-23,changed,changed,"On Linux and Android, the protection to mitiga...",2
3,4.0,Mozilla Firefox,86.0,2021-02-23,other,developer,Inactive CSS tool is now showing a warning whe...,3
4,5.0,Mozilla Firefox,85.0.2,2021-02-09,fixed,fixed,Fixed a deadlock during startup (bug 1679933),1


In [6]:
!pip install simpletransformers

     |████████████████████████████████| 249 kB 33.6 MB/s 
     |████████████████████████████████| 4.2 MB 51.4 MB/s 
     |████████████████████████████████| 6.6 MB 63.9 MB/s 
     |████████████████████████████████| 1.8 MB 54.7 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 63.0 MB/s 
     |████████████████████████████████| 10.1 MB 33.7 MB/s 
     |████████████████████████████████| 342 kB 77.9 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 66.5 MB/s 
     |████████████████████████████████| 181 kB 74.2 MB/s 
     |████████████████████████████████| 145 kB 71.9 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 136 kB 53.9 MB/s 
     |████████████████████████████████| 1.1 MB 51.5 MB/s 
     |████████████████████████████████| 212 kB 75.6 MB/s 
     |████████████████████████████████| 127 kB 79.3 MB/s 
     |█████████████

In [7]:
from simpletransformers.classification import ClassificationModel

In [28]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)

In [29]:
train=train[["Release_Note","labels"]]
test=test[["Release_Note","labels"]]

In [30]:
train["Release_Note"]=train["Release_Note"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

In [31]:
train["Release_Note"].head()

264    Column choice and sort order in "Search Messag...
615    Fixed a bug where log files wouldn't be create...
329    Due to changes in the Mozilla platform profile...
342                               Various security fixes
394    Default to using SSL for XMPP and IRC. This mi...
Name: Release_Note, dtype: object

In [33]:
train["labels"].head()

264    1
615    2
329    4
342    1
394    2
Name: labels, dtype: int64

In [49]:
model = ClassificationModel('bert', 'bert-base-uncased',
                            num_labels=5, use_cuda=False,
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 
                                  'num_train_epochs': 10, 
                                    "train_batch_size": 64 , "fp16":False, 
                                     "output_dir": "./drive/MyDrive/Bert/bert_model4"})

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [50]:
model.train_model(train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/640 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/10 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/10 [00:00<?, ?it/s]

(100, 0.6601109768450261)

In [51]:
result, model_outputs, wrong_predictions = model.eval_model(test)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/160 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/20 [00:00<?, ?it/s]

In [52]:
predictions = model_outputs.argmax(axis=1)

In [53]:
actuals = test.labels.values


In [54]:
predictions[:]

array([2, 2, 1, 0, 3, 2, 4, 2, 1, 0, 1, 4, 1, 1, 4, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 4, 3, 0, 1, 1, 1, 4, 3, 0, 1, 1, 2, 2, 1, 0, 1, 1, 0, 1, 1,
       2, 2, 0, 2, 2, 2, 1, 1, 2, 4, 1, 0, 3, 4, 0, 1, 2, 1, 1, 1, 1, 2,
       1, 1, 4, 1, 1, 4, 1, 0, 1, 3, 4, 1, 2, 1, 1, 0, 3, 1, 2, 1, 1, 2,
       4, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 0, 1, 1, 1, 0, 3, 3, 0, 1, 2, 4,
       1, 0, 3, 0, 2, 3, 0, 1, 0, 4, 1, 1, 1, 0, 1, 2, 2, 1, 0, 2, 2, 0,
       2, 1, 1, 0, 0, 1, 2, 3, 3, 0, 2, 0, 3, 2, 1, 1, 2, 1, 2, 0, 1, 0,
       2, 1, 1, 1, 0, 2])

In [55]:
actuals[:]

array([2, 1, 1, 0, 3, 2, 4, 2, 1, 0, 1, 4, 2, 1, 4, 1, 1, 0, 2, 0, 2, 1,
       2, 2, 1, 3, 2, 1, 0, 1, 4, 3, 0, 1, 1, 0, 2, 1, 1, 1, 1, 0, 1, 1,
       2, 1, 0, 2, 2, 1, 4, 1, 2, 4, 1, 0, 3, 4, 0, 1, 2, 2, 1, 1, 1, 2,
       1, 1, 4, 1, 1, 4, 1, 0, 1, 3, 4, 1, 2, 1, 1, 3, 3, 1, 1, 1, 1, 0,
       4, 0, 1, 2, 2, 0, 2, 2, 2, 1, 2, 1, 1, 1, 1, 0, 3, 1, 0, 1, 2, 4,
       1, 0, 3, 0, 2, 3, 0, 1, 3, 4, 1, 1, 1, 3, 1, 2, 2, 1, 0, 2, 2, 2,
       2, 1, 1, 0, 0, 1, 3, 0, 3, 0, 2, 0, 3, 2, 1, 1, 2, 1, 1, 2, 1, 0,
       2, 1, 1, 1, 0, 2])

In [56]:
from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

0.80625

In [57]:
actual = np.array(actuals[:])
predict = np.array(predictions[:])

In [58]:
import sklearn.metrics as mt
mt.confusion_matrix(actuals, predictions)

array([[20,  3,  4,  1,  0],
       [ 2, 55,  7,  1,  1],
       [ 3,  4, 31,  0,  0],
       [ 3,  0,  1, 11,  0],
       [ 0,  1,  0,  0, 12]])

In [59]:
print(mt.classification_report(actual, predict))

              precision    recall  f1-score   support

           0       0.71      0.71      0.71        28
           1       0.87      0.83      0.85        66
           2       0.72      0.82      0.77        38
           3       0.85      0.73      0.79        15
           4       0.92      0.92      0.92        13

    accuracy                           0.81       160
   macro avg       0.82      0.80      0.81       160
weighted avg       0.81      0.81      0.81       160



In [60]:
örnek = test.iloc[56]['Release_Note']
print(örnek)

QSV Encoder: Improved performance when running OBS on an Intel GPU and simplified various options [brittneysclark],The cursor will now change to reflect your action (dragging edges or sources) when editing items in the preview [WizardCM],Improved the interface of transitions dock [cg2121],Increased the update rate of the level meters in the audio mixer from 30Hz to 60Hz for improved smoothness and responsiveness [dodgepong]


In [63]:
tahmin=model.predict([örnek])

if tahmin[0] ==0:
  print("new")
elif tahmin[0]==1:
  print("fixed")
elif tahmin[0]==2:
  print("changed")
elif tahmin[0]==3:
  print("other")
else:
  print("unresolved")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

other
